In [1]:
import re
import pickle
import json
import pandas as pd

from library.common import Core
from pathlib import Path
cwd = Path.cwd()
prediction_horizon = 4
core = Core()
models = core.all_features

In [2]:
models_path = cwd.parent/'models'
models = list(models_path.glob('validation*.pkl'))

In [3]:
models

[PosixPath('/Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/validation-model-Global-energy_intensity.pkl'),
 PosixPath('/Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/validation-model-Cluster 2-trade_openness.pkl'),
 PosixPath('/Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/validation-model-Cluster 2-co2.pkl'),
 PosixPath('/Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/validation-model-Cluster 0-renewable_energy_consumption_share.pkl'),
 PosixPath('/Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/validation-model-Cluster 2-constant_gdp_per_capita.pkl'),
 PosixPath('/Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/validation-model-Cluster 0-energy_intensity.pkl'),
 PosixPath('/Users/koonleong/Documents/

In [4]:
root_dir = cwd.parent
data_file= r'data/processed/params_forecast_validation.json'
last_data_year_file = root_dir/data_file
file_path = last_data_year_file.as_posix()

In [5]:

with open(file_path, 'r') as datapoint:
    last_data_year = json.load(datapoint)

year_labels = [n for n in range(last_data_year.get('last_period') + 1,
                                last_data_year.get('last_period') + 1 +
                                prediction_horizon, 1)]
print('FORECAST LABELS: ', year_labels)

FORECAST LABELS:  [2017, 2018, 2019, 2020]


In [6]:
# regex = r"-([^-]\w*[^-])-([^-]\w*[^\.])"
regex = r"validation-model-([a-zA-Z0-9|\s]*)-([\w\d]*_{0,1}[\w\d]*)"

forecasts = []
for model in models:
    # print('Processing:', model)
    fname = model.name
    matches = re.search(regex, fname)

    if matches:
        region = matches.group(1)
        feature = matches.group(2)

        print(region, feature)
        print(model.as_posix())

        with open(model.as_posix(), 'rb') as pkl:
            optimal = pickle.load(pkl)

            f = optimal.predict(prediction_horizon)
            print('Forecast ', f)
            fc = zip(year_labels, f)
            forecast_df = pd.DataFrame(fc)
            forecast_df.columns = ['year', 'forecasts']
            forecasts.append([region, feature, forecast_df.to_numpy()])


Global energy_intensity
/Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/validation-model-Global-energy_intensity.pkl
Forecast  [36.99111336 36.11764673 35.2441801  34.37071347]
Cluster 2 trade_openness
/Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/validation-model-Cluster 2-trade_openness.pkl
Forecast  [7083.03617674 7187.62250901 7292.20884129 7396.79517356]
Cluster 2 co2
/Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/validation-model-Cluster 2-co2.pkl
Forecast  [18168.40708943 18373.47917887 18578.5512683  18783.62335774]
Cluster 0 renewable_energy_consumption_share
/Users/koonleong/Documents/UM/UMMADS/Courses - Current/SIADS697-Capstone/power_emissions/models/validation-model-Cluster 0-renewable_energy_consumption_share.pkl
Forecast  [4941.83103679 4941.83103679 4941.83103679 4941.83103679]
Cluster 2 constant_gdp_per_capita
/Users/koonleo

In [7]:
fc_path = cwd.parent/r'data/processed/ts_forecast_validation.pkl'
with open(fc_path, 'wb') as pkl:
    pickle.dump(forecasts, pkl)

print("Done with Forecasts")

Done with Forecasts


In [8]:
# Code to read the forecast
from pathlib import Path
cwd = Path.cwd()
fc_path = cwd.parent/r'data/processed/ts_forecast_validation.pkl'
with open(fc_path, 'rb') as fc_file:
    read_fc = pickle.load(fc_file)
print(read_fc)

[['Global', 'energy_intensity', array([[2017.        ,   36.99111336],
       [2018.        ,   36.11764673],
       [2019.        ,   35.2441801 ],
       [2020.        ,   34.37071347]])], ['Cluster 2', 'trade_openness', array([[2017.        , 7083.03617674],
       [2018.        , 7187.62250901],
       [2019.        , 7292.20884129],
       [2020.        , 7396.79517356]])], ['Cluster 2', 'co2', array([[ 2017.        , 18168.40708943],
       [ 2018.        , 18373.47917887],
       [ 2019.        , 18578.5512683 ],
       [ 2020.        , 18783.62335774]])], ['Cluster 0', 'renewable_energy_consumption_share', array([[2017.        , 4941.83103679],
       [2018.        , 4941.83103679],
       [2019.        , 4941.83103679],
       [2020.        , 4941.83103679]])], ['Cluster 2', 'constant_gdp_per_capita', array([[   2017.        , 1731687.80863576],
       [   2018.        , 1748889.37770543],
       [   2019.        , 1766090.94677511],
       [   2020.        , 1783292.51584478]